In [1]:
import json
import minsearch

In [2]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
    

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=['question','text','section'],
                keyword_fields=['course']
)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
index.fit(documents)

In [8]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### Now, add the LLM component

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [ ]:
#testing that api key is in environment
import os
#os.environ

In [13]:
q

'the course has already started, can I still enroll?'

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":"course has already started, is it too late to join?"}]
)

In [15]:
response

ChatCompletion(id='chatcmpl-BeOSGcq4haorW0qFYmYKoo0T47php', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether it's too late to join a course depends on several factors, including the policies of the institution offering the course, the course format, and how far into the course you are. Here are a few steps you can take:\n\n1. **Contact the Instructor or Institution**: Reach out to the course instructor or the administrative office to inquire about joining late. They can provide information on the possibility and any necessary steps you may need to take.\n\n2. **Review Course Materials**: If possible, review any available syllabus or course materials to assess how much content you’d need to catch up on.\n\n3. **Consider the Course Structure**: For self-paced or online courses, it might be easier to join late as you can work through the materials at your own pace. For in-person or cohort-based online courses, there might be more

In [ ]:
# response.choices[0].message.content

### This is only using our knowledge base, need to connect to GPT

In [28]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
"""

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [33]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [34]:
prompt = prompt_template.format(question=q, context=context).strip()

In [35]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?
CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

### Now ready for GPT

In [38]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)

In [37]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### the next phase is to simplify with functions

In [50]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [43]:
def build_prompt(query,search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [44]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user","content":prompt}]
    )
    
    return response.choices[0].message.content    

In [45]:
query = 'the course already started, can I still enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [46]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [47]:
rag('how do I run kafka?')

'NONE'

In [51]:
rag('do we need to install docker?')


'NONE'

### Now working with Elastic Search

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
es_client.info

<bound method Elasticsearch.info of <Elasticsearch(['http://localhost:9200'])>>

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)

In [ ]:
documents[0]

In [ ]:
for doc in documents:
    es_client.index(index=index_name, document=doc)